In [29]:
import split_op as split
import numpy as np
from scipy.special import roots_legendre, lpmv
from tqdm import tqdm


from potential import load_potential

data_path = "data/"

KELVIN = 3.1668105e-6
U = 1822.88839

def centrifugal(r_points, j_tot: int, omega: int, mass_u: float):
    return (j_tot * (j_tot + 1) - 2 * omega * omega) / (2 * mass_u * U * np.power(r_points, 2)) 

In [30]:
from typing import Callable

def prepare(j_init: int, 
    omega_init: int, 
    j_tot: int, 
    time_step: float = 400, 
    steps_no: float = 360, 
    r_start: float = 50 / 1024, 
    r_end: float = 50, 
    r_no: int = 1024, 
    polar_no: int = 160, 
    mass_u: float = 15.1052848671,
    energy_kelvin: float = 3700, 
    rot_const = 9.243165268327e-7,
    wave_r0: float = 30,
    wave_r_sigma: float = 0.6,
    animation: bool = False,
    frames: int = 60,
    wave_prefix: str = "standard",
    potential_path = "potentials/",
    transform_xpi: Callable[[float, float], float] = None,
    transform_bsigma: Callable[[float, float], float] = None,
    transform_api: Callable[[float, float], float] = None,
    transform_potential: Callable[[float, float], float] = None,
) -> split.Propagation:
    """
    Prepares the split operator propagation of the Ne OCS problem.

    :j_init: initial angular momentum of the OCS molecule
    :omega_init: initial body fixed projection of the OCS angular momentum equal to projection of the total angular momentum
    :j_tot: total angular momentum of the system
    :time_step: time step in the Hartree units of the propagation step
    :steps_no: number of steps in the propagation
    :r_start: starting value of the radial grid
    :r_end: ending value of the radial grid
    :r_no: number of points of the radial grid
    :polar_no: number of points of the angular grid
    :mass_u: reduced mass of the Ne OCS in u units
    :energy_kelvin: energy of the collision in Kelvin units
    :rot_const: rotational constant of the OCS molecule in Hartree units
    :wave_r0: initial radial position of the wave funciton
    :wave_r_sigma: initial radial width of the wave funciton
    :animation: whether to save animations of the wave function
    :frames: number of frames of the animation
    :wave_prefix: prefix of the wave animation filename to be saved
    :wave_legendre_prefix: prefix of the wave animation in the legendre basis filename to be saved
    :potential_path: path to the potential data
    :transform_xpi: custom transformation to the XPi gamma potential
    :transform_bsigma: custom transformation to the BSigma gamma potential
    :transform_api: custom transformation to the APi gamma potential
    :transform_potential: custom transformation to the interatomic potential
    """

    assert j_tot >= omega_init
    assert j_init >= omega_init

    ############################ grids, wave function creation ################################
    
    time_grid = split.TimeGrid(time_step, steps_no)
    r_grid = split.Grid.linear_continuos("r", r_start, r_end, r_no, 0)
    
    polar_points, weights = roots_legendre(polar_no)
    polar_points = np.flip(np.arccos(polar_points))
    weights = np.flip(weights)
    
    polar_grid = split.Grid.custom("theta", polar_points, weights, 1)

    r_points = np.array(r_grid.points())
    momentum = np.sqrt(2 * mass_u * U * energy_kelvin * KELVIN)

    wave_r_init = np.array([split.gaussian_distribution(r_points[i], wave_r0, wave_r_sigma, momentum) for i in range(r_no)])
    wave_polar_init = lpmv(omega_init, j_init, np.cos(polar_points))

    wave_init = np.outer(wave_r_init, wave_polar_init)
    wave_function = split.WaveFunction(wave_init.flatten(), [r_grid, polar_grid])

    ############################ operation creation ################################

    r_mesh, polar_mesh = np.meshgrid(r_points, polar_points)

    potential = load_potential(potential_path, "potential.dat", r_grid, polar_grid, 5, 5, False)
    if transform_potential is not None:
        potential = np.multiply(potential, transform_potential(r_mesh, polar_mesh))

    centrifugal_potential = centrifugal(r_points, j_tot, omega_init, mass_u)
    centrifugal_potential = np.broadcast_to(np.expand_dims(centrifugal_potential, 1), (r_no, polar_no))

    bsigma_gamma = load_potential(potential_path, "BSigma_gamma.dat", r_grid, polar_grid, 5, 5, True)
    if transform_bsigma is not None:
        bsigma_gamma = np.multiply(bsigma_gamma, transform_bsigma(r_mesh, polar_mesh))

    api_gamma = load_potential(potential_path, "APi_gamma.dat", r_grid, polar_grid, 5, 5, True)
    if transform_api is not None:
        api_gamma = np.multiply(api_gamma, transform_api(r_mesh, polar_mesh))

    potential = potential + centrifugal_potential + complex(0, -0.5) * (bsigma_gamma + api_gamma)
    potential_with_bsigma_prop = split.complex_n_dim_into_propagator(potential.shape, potential.flatten(), time_grid)
    potential_with_bsigma_prop.set_loss_checked(split.LossChecker("bsigma"))

    xpi_gamma = load_potential(potential_path, "XPi_gamma.dat", r_grid, polar_grid, 5, 3, True)
    if transform_xpi is not None:
        xpi_gamma = np.multiply(xpi_gamma, transform_xpi(r_mesh, polar_mesh))

    xpi_gamma = complex(0, -0.5) * xpi_gamma
    xpi_gamma_prop = split.complex_n_dim_into_propagator(xpi_gamma.shape, xpi_gamma.flatten(), time_grid)
    xpi_gamma_prop.set_loss_checked(split.LossChecker("xpi"))

    leak_control = split.LeakControl(split.LossChecker("leak control"))
    dumping_border = split.BorderDumping(5., 1., r_grid)

    angular_transformation = split.legendre_transformation(polar_grid)

    shape, angular_kinetic_op = split.rotational_hamiltonian(r_grid, polar_grid, mass_u, rot_const)
    angular_prop = split.n_dim_into_propagator(shape, angular_kinetic_op, time_grid)

    fft_transformation = split.FFTTransformation(r_grid, "r momentum")

    kinetic_op = split.kinetic_hamiltonian(r_grid, mass_u, energy_kelvin)
    kinetic_prop = split.one_dim_into_propagator(kinetic_op, r_grid, time_grid, step = "full")

    ################################ populating operation stack ####################################

    operation_stack = split.OperationStack()
    potential_with_bsigma_prop.add_operation(operation_stack)
    xpi_gamma_prop.add_operation(operation_stack)

    if animation:
        wave_saver = split.WaveFunctionSaver(data_path, f"{wave_prefix}_wave_animation", time_grid, r_grid, polar_grid, frames)
        wave_saver.add_operation(operation_stack)

    dumping_border.add_operation(operation_stack)
    leak_control.add_operation(operation_stack)

    angular_transformation.add_operation(operation_stack, True)

    if animation:
        angular_grid = angular_transformation.transformed_grid()
        wave_legendre_saver = split.StateSaver(data_path, f"{wave_prefix}_angular_animation", time_grid, angular_grid, frames)
        wave_legendre_saver.add_operation(operation_stack)

    angular_prop.add_operation(operation_stack)

    fft_transformation.add_operation(operation_stack, True)
    kinetic_prop.add_operation(operation_stack)

    ################################ propagation creation ####################################

    propagation = split.Propagation()
    propagation.set_wave_function(wave_function)
    propagation.set_time_grid(time_grid)
    propagation.set_operation_stack(operation_stack)

    return propagation

class CumulativeLosses:
    def __init__(self, j_init: int, energy_kelvin: float = 3700) -> None:
        self.bsigma_losses = []
        self.xpi_losses = []

        self.j_totals = [j_init + np.ceil(i * 5.5 * np.sqrt(energy_kelvin / 2000)) for i in range(50)]
    
    def extract_loss(self, propagation: split.Propagation) -> None:
        losses = propagation.get_losses()

        self.bsigma_losses.append(losses[0])
        self.xpi_losses.append(losses[1])

    def save_losses(self, filename: str) -> None:
        combined = np.vstack((self.j_totals, self.bsigma_losses, self.xpi_losses)).transpose()
        np.savetxt(f"{data_path}{filename}.dat", combined, delimiter=" ", header="j_total\tbsigma_loss\txpi_loss")

# Animations

In [4]:
print("j init: 0, omega init: 0, j_tot: 150")
propagation = prepare(0, 0, 150, animation=True) # time step as if was halfed as oposed to the implementation in rust ?

propagation.propagate()
print(propagation.get_losses())
propagation.save_savers()

[0.4647287667794322, 0.19602354128894028]


In [5]:
print("j init: 0, omega init: 0, j_tot: 150")
propagation = prepare(0, 0, 150, animation=True, rot_const=9.243165268327e-9, wave_prefix="small_rot_scaling")

propagation.propagate()
print(propagation.get_losses())
propagation.save_savers()

[0.4944627253054539, 0.2023044273100963]


In [6]:
print("j init: 0, omega init: 0, j_tot: 150")
propagation = prepare(0, 0, 150, animation=True, rot_const=9.243165268327e-5, wave_prefix="big_rot_scaling")

propagation.propagate()
print(propagation.get_losses())
propagation.save_savers()

[0.47047267929405034, 0.1857722951025123]


# Single losses

In [10]:
print("j init: 0, omega init: 0, j_tot: 150")
propagation = prepare(0, 0, 150)
propagation.propagate()
print(propagation.get_losses())

print("j init: 1, omega init: 0, j_tot: 150")
propagation = prepare(1, 0, 150)
propagation.propagate()
print(propagation.get_losses())

print("j init: 1, omega init: 1, j_tot: 150")
propagation = prepare(1, 1, 150)
propagation.propagate()
print(propagation.get_losses())

j init: 0 omega init: 0 j total: 150
[0.4647287667794322, 0.19602354128894028]
j init: 1 omega init: 0 j total: 150
[0.5148534748051525, 0.17456010613315387]
j init: 1 omega init: 1 j total: 150
[0.4400325245720494, 0.20649488661234802]


# Single cross sections

In [33]:
j_init = 0
omega_init = 0
energy_kelvin = 3700
losses = CumulativeLosses(j_init, energy_kelvin)

for j_tot in tqdm(losses.j_totals):
    propagation = prepare(j_init, omega_init, j_tot, energy_kelvin=energy_kelvin)
    propagation.propagate()
    losses.extract_loss(propagation)
losses.save_losses(f"losses_{energy_kelvin}_{j_init}_{omega_init}")

100%|██████████| 50/50 [01:29<00:00,  1.78s/it]


In [ ]:
j_init = 1
omega_init = 0
energy_kelvin = 3700
losses = CumulativeLosses(j_init, energy_kelvin)

for j_tot in tqdm(losses.j_totals):
    propagation = prepare(j_init, omega_init, j_tot, energy_kelvin=energy_kelvin)
    propagation.propagate()
    losses.extract_loss(propagation)
losses.save_losses(f"losses_{energy_kelvin}_{j_init}_{omega_init}")

In [ ]:
j_init = 1
omega_init = 1
energy_kelvin = 3700
losses = CumulativeLosses(j_init, energy_kelvin)

for j_tot in tqdm(losses.j_totals):
    propagation = prepare(j_init, omega_init, j_tot, energy_kelvin=energy_kelvin)
    propagation.propagate()
    losses.extract_loss(propagation)
losses.save_losses(f"losses_{energy_kelvin}_{j_init}_{omega_init}")

# Cross sections - energy dependence 

In [34]:
energies = [100, 500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000]

In [35]:
for energy_kelvin in energies:
    print("energy", energy_kelvin)
    j_init = 0
    omega_init = 0
    losses = CumulativeLosses(j_init, energy_kelvin)

    for j_tot in tqdm(losses.j_totals):
        propagation = prepare(j_init, omega_init, j_tot, energy_kelvin=energy_kelvin)
        propagation.propagate()
        losses.extract_loss(propagation)
    losses.save_losses(f"losses_{energy_kelvin}_{j_init}_{omega_init}")

energy 100


100%|██████████| 50/50 [01:28<00:00,  1.76s/it]


energy 500


100%|██████████| 50/50 [01:29<00:00,  1.79s/it]


energy 1000


...

100%|██████████| 50/50 [01:28<00:00,  1.76s/it]


energy 4500


100%|██████████| 50/50 [01:28<00:00,  1.76s/it]


energy 5000


100%|██████████| 50/50 [01:27<00:00,  1.74s/it]


In [36]:
for energy_kelvin in energies:
    print("energy", energy_kelvin)
    j_init = 1
    omega_init = 0
    losses = CumulativeLosses(j_init, energy_kelvin)

    for j_tot in tqdm(losses.j_totals):
        propagation = prepare(j_init, omega_init, j_tot, energy_kelvin=energy_kelvin)
        propagation.propagate()
        losses.extract_loss(propagation)
    losses.save_losses(f"losses_{energy_kelvin}_{j_init}_{omega_init}")

energy 100


100%|██████████| 50/50 [01:26<00:00,  1.72s/it]


energy 500


100%|██████████| 50/50 [01:26<00:00,  1.73s/it]


energy 1000


...

100%|██████████| 50/50 [01:26<00:00,  1.72s/it]


energy 4500


100%|██████████| 50/50 [01:26<00:00,  1.73s/it]


energy 5000


100%|██████████| 50/50 [01:26<00:00,  1.73s/it]


In [37]:
for energy_kelvin in energies:
    print("energy", energy_kelvin)
    j_init = 1
    omega_init = 1
    losses = CumulativeLosses(j_init, energy_kelvin)

    for j_tot in tqdm(losses.j_totals):
        propagation = prepare(j_init, omega_init, j_tot, energy_kelvin=energy_kelvin)
        propagation.propagate()
        losses.extract_loss(propagation)
    losses.save_losses(f"losses_{energy_kelvin}_{j_init}_{omega_init}")

energy 100


100%|██████████| 50/50 [01:27<00:00,  1.76s/it]


energy 500


100%|██████████| 50/50 [01:27<00:00,  1.75s/it]


energy 1000


...

100%|██████████| 50/50 [01:26<00:00,  1.73s/it]


energy 4500


100%|██████████| 50/50 [01:26<00:00,  1.73s/it]


energy 5000


100%|██████████| 50/50 [01:27<00:00,  1.75s/it]


In [42]:
import inspect

print(inspect.signature(prepare).parameters.get("energy_kelvin").default)


3700
